## A better way to manipulate data

This is an API proposal to access data

Dataframes would have to ojects, rows and columns. This will have the same access methods:

* create
* read
* move
* update
* delete

For example

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from optimus import *


from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType

sc = SparkSession.builder.getOrCreate()
# Create optimus
op = Optimus(sc)

df = op.create.dataframe([
                ("  I like     fish  ", 1, "dog", "housé" ),
                ("    zombies", 2, "cat", "tv"),
                ("simpsons   cat lady", 2, "frog", "table"),
                (None, 3, "eagle", "glass")
            ],
            [
                ("words", StringType(), True),
                ("num", IntegerType(), True),
                ("animals", StringType(), True),
                ("thing", StringType(), True)
            ])

df.show()

Using a created Spark Session...
Done.
+-------------------+---+-------+-----+
|              words|num|animals|thing|
+-------------------+---+-------+-----+
|  I like     fish  |  1|    dog|housé|
|            zombies|  2|    cat|   tv|
|simpsons   cat lady|  2|   frog|table|
|               null|  3|  eagle|glass|
+-------------------+---+-------+-----+



In [10]:
print("*** Load files operations")
op.load.csv()

print ("*** Rows CRUD operations")

# Rows CRUD Operations
op.rows.create()
op.rows.select()
op.rows.update()
op.rows.delete()


op.rows.apply()

print ("*** Columns CRUD operations")

# Column Operation
op.columns.create()
op.columns.select(by_index = 1)

*** Load files operations
CSV file Loaded
*** Rows CRUD operations
Row Created
Row updated with new content
Row Deleted
Row Apply
*** Columns CRUD operations
Column created
Select column by index


## Pandas comparision
Pandas vs Spark
https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/

In [83]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.arange(12).reshape(3,4),columns=['A', 'B', 'C', 'D'])

# Drop col
df.drop(['B', 'C'], axis=1)
# or df.drop(columns=['B', 'C'])

,A,D
0,0,3
1,4,7
2,8,11


In [80]:
# Drop by index
df.drop([0, 1])

,A,B,C,D
2,8,9,10,11


In [ ]:
# Spark
df.drop('B')
df.drop('C')#

# Multiple olumns
columns_to_drop = ['A', 'B']
df.drop(*columns_to_drop)

# Optimus